# S3W10D5: 给 RAG 装上“精排大脑” (Implementing Re-ranker)


## 1. 理论知识讲解 (The "Why" Recap)

在动手写代码前，再次确认**“漏斗模式” (Funnel Architecture)** 的流程：

1. **Recall (海选)**: 向量库（Bi-Encoder）。速度快，负责从 10000 条数据里捞出 Top-10。它看重的是“大概长得像”。
2. **Rerank (精选)**: 重排序模型（Cross-Encoder）。速度慢，负责把这 10 条数据逐字逐句读一遍，重新打分，选出 Top-3。它看重的是“逻辑对不对”。

**我们要使用的模型**: `BAAI/bge-reranker-base`。

* 这是一个在中文语境下表现极好的模型。
* 它的工作方式是：给它 `(Query, Document)` 对，它直接输出一个分数（Score），分数越高越相关。


## 2. 代码实现 (Project Code)

请将以下代码写入 `src/rag/reranker.py`。

我们需要使用 HuggingFace 的 `transformers` 库。

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from typing import List, Tuple

class RerankClient:
    def __init__(self, model_name: str = "BAAI/bge-reranker-base"):
        """
        初始化重排序模型 (Cross-Encoder)
        """
        print(f"Loading Reranker model: {model_name}...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.model.eval() # 开启推理模式
        
        # 如果有 GPU，转到 GPU
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model.to(self.device)
        print(f"Reranker loaded on {self.device}")

    def rank(self, query: str, documents: List[str], top_k: int = 3) -> List[Tuple[str, float]]:
        """
        对文档列表进行重排序
        Args:
            query: 用户的问题
            documents: 向量检索回来的候选文档列表 (Strings)
            top_k: 返回前 k 个最好的
        Returns:
            List of (document_content, score), sorted by score descending
        """
        if not documents:
            return []

        # 1. 构建输入对: [[Query, Doc1], [Query, Doc2], ...]
        pairs = [[query, doc] for doc in documents]

        # 2. Tokenize (自动处理 [CLS] 和 [SEP])
        with torch.no_grad():
            inputs = self.tokenizer(
                pairs, 
                padding=True, 
                truncation=True, 
                return_tensors='pt', 
                max_length=512
            ).to(self.device)

            # 3. 计算分数 (Forward Pass)
            # BGE Reranker 输出的是 logits，数值范围不限，越大越好
            scores = self.model(**inputs, return_dict=True).logits.view(-1,).float()

        # 4. 排序
        # 将文档和分数打包
        results = list(zip(documents, scores.cpu().tolist()))
        
        # 按分数从高到低排序
        results.sort(key=lambda x: x[1], reverse=True)

        return results[:top_k]

# 简单的测试代码
if __name__ == "__main__":
    client = RerankClient()
    q = "这就去办"
    docs = ["好的，马上处理", "我不明白你的意思", "苹果是一种水果", "这就去办的相关政策"]
    print(client.rank(q, docs))


Loading Reranker model: BAAI/bge-reranker-base...


'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /BAAI/bge-reranker-base/resolve/main/tokenizer_config.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x754d96ce8af0>: Failed to establish a new connection: [Errno 101] Network is unreachable'))"), '(Request ID: 6c01cdb5-e741-4264-b8f1-00f371069aca)')' thrown while requesting HEAD https://huggingface.co/BAAI/bge-reranker-base/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].


> **注意**: 第一次运行会自动下载模型权重（约 1GB），请确保网络通畅。

## 3. 深度理论 (Integration Logic)

在你的 `S3W10D5_Reranker_Impl.ipynb` 中，我们要模拟 **Vector Search + Rerank** 的全流程。

请运行以下代码，直观感受 Rerank 的威力：

In [1]:
import sys
import os
# 把项目根目录加入 path，确保能 import src
sys.path.append(os.path.abspath('../../'))

from src.rag.reranker import RerankClient

# 1. 初始化
reranker = RerankClient("BAAI/bge-reranker-base")

# 2. 构造一个刁钻的场景
# Query: 询问关于 "熊猫" (Panda) 的饮食，但是我们要设置陷阱
query = "熊猫不吃什么？"

# 3. 模拟向量检索 (Vector Search) 的结果
# 向量检索通常基于关键词匹配，所以只要包含 "熊猫"、"吃" 的文档都会排在前面
# 假设 Vector DB 返回了以下 Top-5：
retrieved_docs = [
    "大熊猫主要以竹子为食，每天要吃很多。",  # Doc A: 讲吃的 (相关性高，但没直接回答不吃什么)
    "熊猫有时候也会吃一些肉类，比如竹鼠。",  # Doc B: 讲吃的
    "小明很喜欢吃熊猫饼干。",                # Doc C: 干扰项 (字面相似)
    "铁块和石头是绝对不能吃的。",             # Doc D: 讲不吃的 (但没提熊猫，向量距离可能远)
    "虽然熊猫是食肉目，但它们基本不吃肉，主要吃素。" # Doc E: 正确答案 (包含逻辑否定)
]

print(f"Query: {query}\n")
print("--- Before Rerank (Naive Order) ---")
for i, doc in enumerate(retrieved_docs):
    print(f"[{i+1}] {doc}")

# 4. 执行重排序
print("\n--- After Rerank (Cross-Encoder) ---")
reranked_results = reranker.rank(query, retrieved_docs, top_k=3)

for i, (doc, score) in enumerate(reranked_results):
    print(f"Rank {i+1} (Score: {score:.4f}): {doc}")

Loading Reranker model: BAAI/bge-reranker-base...


'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /BAAI/bge-reranker-base/resolve/main/tokenizer_config.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x77549eb5c100>: Failed to establish a new connection: [Errno 101] Network is unreachable'))"), '(Request ID: a5164c1e-33e3-4d02-9f95-8cb82439f482)')' thrown while requesting HEAD https://huggingface.co/BAAI/bge-reranker-base/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /BAAI/bge-reranker-base/resolve/main/tokenizer_config.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x77549e934f10>: Failed to establish a new connection: [Errno 101] Network is unreachable'))"), '(Request ID: f47fbabe-a84b-4060-a1c1-6ee535145646)')' thrown while requesting HEAD https://huggingface.co/BAAI/bge-reranker-base/resolve/m

KeyboardInterrupt: 

### 预期结果分析

* **Doc C (熊猫饼干)**: 在向量检索中可能排名靠前（因为字面重合度高），但 Reranker 应该给它打极低分。
* **Doc E (不吃肉)**: 这句话对于 Query "不吃什么" 是最佳答案。Reranker 应该把它排到 **Rank 1**。
* 如果你的代码能把 **Doc E** 捞到第一名，恭喜你，你的 RAG 系统具备了初步的逻辑理解能力！





## 4. 模拟面试问答 (Interview Q&A)

**Q1: 既然 Cross-Encoder 效果这么好，为什么不直接对全库做 Cross-Encoder？**

> **参考答案**: 这是一个**时间复杂度**的问题。
> * Cross-Encoder 需要把 Query 和每一个 Document 拼接在一起进 BERT，计算量巨大。假设全库有 100万 文档，处理一个 Query 可能需要几小时。
> * Bi-Encoder (向量检索) 可以预先计算好 Document 向量，检索时只是做向量内积（ANN），能在毫秒级完成。
> * 所以我们采用**“漏斗策略”**：用便宜快速的模型做召回，用昂贵精准的模型做精排。
> 
> 

**Q2: BGE-Reranker 的输入长度限制是多少？如果文档很长怎么办？**

> **参考答案**: BERT 类模型的限制通常是 512 tokens。
> 如果 `Len(Query) + Len(Doc)` 超过 512，通常会被**截断 (Truncation)**。这会导致丢失信息。
> **解决方案**: 在 RAG 的数据处理阶段 (Chunking) 就应该把文档切分成小于 500 tokens 的片段，确保重排序时能看到完整的语义。

## 5. 今日算法练习 (Algorithm)

由于你在树的题目上表现优异，今天我们来一道**“二叉树 + 逻辑推理”**的进阶题，这道题也是 Facebook/Bytedance 的高频题。

**题目**: [LC 236. 二叉树的最近公共祖先 (Lowest Common Ancestor)](https://leetcode.com/problems/lowest-common-ancestor-of-a-binary-tree/)
**难度**: Medium

**核心逻辑 (后序遍历)**:
我们要找节点 `p` 和 `q` 的祖先。
对于当前节点 `root`：

1. 如果在**左子树**找到了 p 或 q？
2. 如果在**右子树**找到了 p 或 q？
3. 如果**左边也有，右边也有** -> 那我（root）就是那个公共祖先！
4. 如果只有一边有 -> 那结果就在那一边，往上传递。

**Action**: 请尝试用递归写出这道题。如果卡住了，想想“向上汇报”的机制。

## 6. 今日任务总结

* [ ] **Code**: 成功运行 `src/rag/reranker.py`，模型能下载并加载。
* [ ] **Test**: 在 Notebook 中看到 Reranker 成功把最符合逻辑的文档排到了第一位。
* [ ] **Algorithm**: 挑战 LCA 问题 (LC 236)。

完成这些后，你的 RAG 就不再是简单的玩具了，它拥有了“精挑细选”的能力。**Day 6** 我们将把这些组件全部串起来，做一个完整的命令行版 AI 助手！